In [10]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import json

class ComparadorInstagram:
    def __init__(self, root):
        self.root = root
        self.root.title("Comparador de Seguidores y Seguidos de Instagram")
        self.root.geometry("800x600")
        
        self.seguidores = []
        self.seguidos = []
        
        self.crear_interfaz()

    def crear_interfaz(self):
        tk.Button(self.root, text="Cargar JSON de Seguidores", command=self.cargar_seguidores).pack(pady=10)
        tk.Button(self.root, text="Cargar JSON de Seguidos", command=self.cargar_seguidos).pack(pady=10)
        tk.Button(self.root, text="Comparar", command=self.comparar).pack(pady=20)
        
        self.resultado_text = scrolledtext.ScrolledText(self.root, height=20)
        self.resultado_text.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

    def cargar_seguidores(self):
        archivo = filedialog.askopenfilename(filetypes=[("Archivos JSON", "*.json")])
        if archivo:
            self.seguidores = self.cargar_datos(archivo)
            messagebox.showinfo("Éxito", f"Se cargaron {len(self.seguidores)} seguidores")

    def cargar_seguidos(self):
        archivo = filedialog.askopenfilename(filetypes=[("Archivos JSON", "*.json")])
        if archivo:
            self.seguidos = self.cargar_datos(archivo)
            messagebox.showinfo("Éxito", f"Se cargaron {len(self.seguidos)} seguidos")

    def cargar_datos(self, archivo):
        try:
            with open(archivo, 'r', encoding='utf-8') as f:
                datos = json.load(f)
            usuarios = []
            if isinstance(datos, dict):
                for clave, valor in datos.items():
                    if isinstance(valor, list):
                        for item in valor:
                            if isinstance(item, dict) and 'string_list_data' in item:
                                for subitem in item['string_list_data']:
                                    if 'value' in subitem:
                                        usuarios.append(subitem['value'])
            elif isinstance(datos, list):
                for item in datos:
                    if isinstance(item, dict) and 'string_list_data' in item:
                        for subitem in item['string_list_data']:
                            if 'value' in subitem:
                                usuarios.append(subitem['value'])
            return usuarios
        except Exception as e:
            messagebox.showerror("Error", f"Error al cargar el archivo: {str(e)}")
            return []

    def comparar(self):
        if not self.seguidores or not self.seguidos:
            messagebox.showwarning("Advertencia", "Por favor, carga ambos archivos JSON primero.")
            return
        
        set_seguidores = set(self.seguidores)
        set_seguidos = set(self.seguidos)
        
        no_te_siguen = set_seguidos - set_seguidores
        no_sigues = set_seguidores - set_seguidos
        
        resultado = "Usuarios que sigues pero no te siguen de vuelta:\n"
        resultado += "\n".join(no_te_siguen) + "\n\n"
        resultado += "Usuarios que te siguen pero no sigues de vuelta:\n"
        resultado += "\n".join(no_sigues)
        
        self.resultado_text.delete(1.0, tk.END)
        self.resultado_text.insert(tk.END, resultado)

def main():
    root = tk.Tk()
    ComparadorInstagram(root)
    root.mainloop()

if __name__ == "__main__":
    main()